In [6]:
from itertools import cycle
from collections import deque

def score_game(num_players, num_marbles, part2=False):
    marbles = iter(range(num_marbles))
    circle = deque([next(marbles)])

    players = cycle(range(num_players))
    scores = [0] * num_players

    for marble in marbles:
        player = next(players)

        if marble % 23 == 0:  # special
            scores[player] += marble
            circle.rotate(7)
            scores[player] += circle.popleft()
        else:
            circle.rotate(-2)
            circle.insert(0, marble)
            
        #if num_marbles < 100:
        #    print(player+1, marble, circle)
    return scores

In [9]:
print(max(score_game(num_players=9, num_marbles=26+1)))
%timeit score_game(num_players=416, num_marbles=71975+1)
print(max(score_game(num_players=416, num_marbles=71975+1)))
print(max(score_game(num_players=416, num_marbles=7197500+1)))

32
34.3 ms ± 232 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
439341
3566801385


In [ ]:
from itertools import cycle
import operator

class Marble(object):
    def __init__(self, value, cw=None, ccw=None):
        self.value = value
        self.cw = cw
        self.ccw = ccw
    def __repr__(self):
        return str(self.value)

class Circle(object):
    def __init__(self):
        self.zero_marble = Marble(0)
        self.current_marble = self.zero_marble
        self.current_marble.cw = self.current_marble.ccw = self.current_marble
    
    def __repr__(self):
        marble = self.zero_marble
        s = repr(self.zero_marble)
        
        while marble.cw != self.zero_marble:
            marble = marble.cw
            s += '-' + repr(marble)
            
        return s
    
def insert(marble, cw_of=None, ccw_of=None): 
    if cw_of:
        cw = cw_of.cw
        ccw = cw_of
        
    elif ccw_of:
        cw = ccw_of
        ccw = ccw_of.ccw

    marble.cw = cw
    marble.ccw = ccw
    ccw.cw = marble
    cw.ccw = marble
    
def pop(marble):
    cw = marble.cw
    ccw = marble.ccw
    marble.ccw.cw = cw
    marble.cw.ccw = ccw
    
    return marble

def rotate(marble, amount=0):
    if amount > 0:
        op = operator.attrgetter('cw')
    else:
        op = operator.attrgetter('ccw')
        
    for i in range(abs(amount)):
        marble = op(marble)
        
    return marble

class Game(object):
    def __init__(self, num_players, max_marble_value):
        self.num_players = num_players
        self.players = cycle(range(num_players))
        
        self.max_marble_value = max_marble_value
        self.circle = None

    def play(self):
        self.circle = Circle()
        scores = [0] * self.num_players
        
        for marble_value in iter(range(1, self.max_marble_value + 1)):
            if marble_value % max(100000, self.max_marble_value//1000) == 0:
                print("{0:.1%}".format(marble_value/self.max_marble_value))

            player = next(self.players)
            if marble_value % 23 == 0:  # special
                scores[player] += marble_value
                removed_marble = pop(rotate(self.circle.current_marble, -7))
                scores[player] += removed_marble.value
                self.circle.current_marble = removed_marble.cw
                continue

            marble = Marble(marble_value)
            insert(marble, cw_of=rotate(self.circle.current_marble, 1))
            self.circle.current_marble = marble
        
        return scores
    
g = Game(num_players=416, max_marble_value=71975)
%timeit g.play()
print(max(g.play()))

g = Game(num_players=416, max_marble_value=7197500)
%timeit g.play()
print(max(g.play()))

188 ms ± 1.91 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
439341
1.4%
2.8%
4.2%
5.6%
6.9%
8.3%
9.7%
11.1%
12.5%
13.9%
15.3%
16.7%
18.1%
19.5%
20.8%
22.2%
23.6%
25.0%
26.4%
27.8%
29.2%
30.6%
32.0%
33.3%
34.7%
36.1%
37.5%
38.9%
40.3%
41.7%
43.1%
44.5%
45.8%
47.2%
48.6%
50.0%
51.4%
52.8%
54.2%
55.6%
57.0%
58.4%
59.7%
61.1%
62.5%
63.9%
65.3%
66.7%
68.1%
69.5%
70.9%
72.2%
73.6%
75.0%
76.4%
77.8%
79.2%
80.6%
82.0%
83.4%
84.8%
86.1%
87.5%
88.9%
90.3%
91.7%
93.1%
94.5%
95.9%
97.3%
98.6%
1.4%
2.8%
4.2%
5.6%
6.9%
8.3%
9.7%
11.1%
12.5%
13.9%
15.3%
16.7%
18.1%
19.5%
20.8%
22.2%
23.6%
25.0%
26.4%
27.8%
29.2%
30.6%
32.0%
33.3%
34.7%
36.1%
37.5%
38.9%
40.3%
41.7%
43.1%
44.5%
45.8%
47.2%
48.6%
50.0%
51.4%
52.8%
54.2%
55.6%
57.0%
58.4%
59.7%
61.1%
62.5%
63.9%
65.3%
66.7%
68.1%
69.5%
70.9%
72.2%
73.6%
75.0%
76.4%
77.8%
79.2%
80.6%
82.0%
83.4%
84.8%
86.1%
87.5%
88.9%
90.3%
91.7%
93.1%
94.5%
95.9%
97.3%
98.6%
1.4%
2.8%
4.2%
5.6%
6.9%
8.3%
9.7%
11.1%
12.5%
13.9%
15.3%
16.7%
18.1%
19.5%
20.8%
22

13.9%
27.8%
41.7%
55.6%
69.5%
83.4%
97.3%
439341


In [9]:
print(max(score_game(num_players=416, num_marbles=71975+1, part2=True)))

439341


In [ ]:
print(max(score_game(num_players=416, num_marbles=(71975*100+1))))

0.1%
0.3%
0.4%
0.6%
0.7%
0.8%
1.0%
1.1%
1.3%
1.4%
1.5%
1.7%
1.8%
1.9%
2.1%
2.2%
2.4%
2.5%
2.6%
2.8%
2.9%
3.1%
3.2%
3.3%
3.5%
3.6%
3.8%
3.9%
4.0%
4.2%
4.3%
4.4%
4.6%
4.7%
4.9%
5.0%
5.1%
5.3%
5.4%
5.6%
5.7%
5.8%
6.0%
6.1%
6.3%
6.4%
6.5%
6.7%
6.8%
6.9%
7.1%
7.2%
7.4%
7.5%
7.6%
7.8%
7.9%
8.1%
8.2%
8.3%
8.5%
8.6%
8.8%
8.9%
9.0%
9.2%
9.3%
9.4%
9.6%
9.7%
9.9%
10.0%
10.1%
10.3%
10.4%
10.6%
10.7%
10.8%
11.0%
11.1%
11.3%
11.4%
11.5%
11.7%
11.8%
11.9%
12.1%
12.2%
12.4%
12.5%
12.6%
12.8%
12.9%
13.1%
13.2%
13.3%
13.5%
13.6%
13.8%
13.9%
14.0%
14.2%
14.3%
14.4%
14.6%
14.7%
14.9%
15.0%
15.1%
15.3%
15.4%
15.6%
15.7%
15.8%
16.0%
16.1%
16.3%
16.4%
16.5%
16.7%
16.8%
17.0%
17.1%
17.2%
17.4%
17.5%
17.6%
17.8%
17.9%
18.1%
18.2%
18.3%
18.5%
18.6%
18.8%
18.9%
19.0%
19.2%
19.3%
19.5%
19.6%
19.7%
19.9%
20.0%
20.1%
20.3%
20.4%
20.6%
20.7%
20.8%
21.0%
21.1%
21.3%
21.4%
21.5%
21.7%
21.8%
22.0%
22.1%
22.2%
22.4%
22.5%
22.6%
22.8%
22.9%
23.1%
23.2%
23.3%
23.5%
23.6%
23.8%
23.9%
24.0%
24.2%
24.3%
24.5%
24.6%
24.7%
24.